In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Reshape
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import random
import glob

In [2]:
EPOCHS = 2000
N_HIDDEN = 128
OBSERVE_LENGTH = 10
PREDICT_LENGTH = 5
FEAT_DIM = 12
LEARNING_RATE = 0.0001
BATCH_SIZE = 64
IMG_DIM = 2048
VAL_RATIO = 0.1
TRAIN_FOLDERS = '/home/dataset/training_observe_{}_predict_{}/*/'.format(OBSERVE_LENGTH, PREDICT_LENGTH)
# TEST_FOLDERS = '/home/dataset/validation_observe_{}_predict_{}/*/'.format(OBSERVE_LENGTH, PREDICT_LENGTH)

MODEL_NAME = 'sslstm-image_epoch_{}_hidden_{}_observe_{}_predict_{}'.format(EPOCHS, N_HIDDEN, OBSERVE_LENGTH, PREDICT_LENGTH)

In [3]:
train_folders = glob.glob(TRAIN_FOLDERS)
random.shuffle(train_folders)
val_num = int(VAL_RATIO * len(train_folders))
val_folders = train_folders[-val_num:]
train_folders = train_folders[:-val_num]

print('train folder num:', len(train_folders))
print('val folder num:', len(val_folders))

train folder num: 18000
val folder num: 2000


In [4]:
import tensorflow.keras as keras

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, shuffle=True):
        'Initialization'
        self.batch_size = BATCH_SIZE
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X_image_0 = np.empty((self.batch_size, OBSERVE_LENGTH, IMG_DIM))
        X_image_1 = np.empty((self.batch_size, OBSERVE_LENGTH, IMG_DIM))
        X_image_2 = np.empty((self.batch_size, OBSERVE_LENGTH, IMG_DIM))
        X_image_3 = np.empty((self.batch_size, OBSERVE_LENGTH, IMG_DIM))
        X_image_4 = np.empty((self.batch_size, OBSERVE_LENGTH, IMG_DIM))
        
        X_feat = np.empty((self.batch_size, OBSERVE_LENGTH, FEAT_DIM))
        y = np.empty((self.batch_size, PREDICT_LENGTH, 3), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            
            image_seq_0 = []
            with open(ID + 'img_path_0.txt', 'r') as f:
                img_path_0 = f.read().split('\n')
            for p in img_path_0:
                image_seq_0.append(np.load(p))
            X_image_0[i,] = np.array(image_seq_0)
            
            image_seq_1 = []
            with open(ID + 'img_path_1.txt', 'r') as f:
                img_path_1 = f.read().split('\n')
            for p in img_path_1:
                image_seq_1.append(np.load(p))
            X_image_1[i,] = np.array(image_seq_1)
            
            image_seq_2 = []
            with open(ID + 'img_path_2.txt', 'r') as f:
                img_path_2 = f.read().split('\n')
            for p in img_path_2:
                image_seq_2.append(np.load(p))
            X_image_2[i,] = np.array(image_seq_2)
            
            image_seq_3 = []
            with open(ID + 'img_path_3.txt', 'r') as f:
                img_path_3 = f.read().split('\n')
            for p in img_path_3:
                image_seq_3.append(np.load(p))
            X_image_3[i,] = np.array(image_seq_3)
            
            image_seq_4 = []
            with open(ID + 'img_path_4.txt', 'r') as f:
                img_path_4 = f.read().split('\n')
            for p in img_path_4:
                image_seq_4.append(np.load(p))
            X_image_4[i,] = np.array(image_seq_4)
                
            X_feat[i,] = np.load(ID + 'X.npy')[:, :-3]
            y[i] = np.load(ID + 'y.npy')

        return [X_image_0, X_image_1, X_image_2, X_image_3, X_image_4, X_feat], y


In [7]:
def build_model():
    opt = optimizers.RMSprop(lr=LEARNING_RATE)
    
    
    feat_input = Input(shape=(OBSERVE_LENGTH, FEAT_DIM))
    img_input_0 = Input(shape=(OBSERVE_LENGTH, IMG_DIM))
    img_input_1 = Input(shape=(OBSERVE_LENGTH, IMG_DIM))
    img_input_2 = Input(shape=(OBSERVE_LENGTH, IMG_DIM))
    img_input_3 = Input(shape=(OBSERVE_LENGTH, IMG_DIM))
    img_input_4 = Input(shape=(OBSERVE_LENGTH, IMG_DIM))
    
    #encoder_feat
    encoder_feat = layers.GRU(N_HIDDEN,
                  input_shape=(OBSERVE_LENGTH, FEAT_DIM),
                  return_sequences=False,
                  stateful=False,
                  dropout=0.2)(feat_input)

    encoder_img_0 = layers.GRU(N_HIDDEN,
                  input_shape=(OBSERVE_LENGTH, IMG_DIM),
                  return_sequences=False,
                  stateful=False,
                  dropout=0.2)(img_input_0)
    
    encoder_img_1 = layers.GRU(N_HIDDEN,
                  input_shape=(OBSERVE_LENGTH, IMG_DIM),
                  return_sequences=False,
                  stateful=False,
                  dropout=0.2)(img_input_1)
    
    encoder_img_2 = layers.GRU(N_HIDDEN,
                  input_shape=(OBSERVE_LENGTH, IMG_DIM),
                  return_sequences=False,
                  stateful=False,
                  dropout=0.2)(img_input_2)
    
    encoder_img_3 = layers.GRU(N_HIDDEN,
                  input_shape=(OBSERVE_LENGTH, IMG_DIM),
                  return_sequences=False,
                  stateful=False,
                  dropout=0.2)(img_input_3)
    
    encoder_img_4 = layers.GRU(N_HIDDEN,
                  input_shape=(OBSERVE_LENGTH, IMG_DIM),
                  return_sequences=False,
                  stateful=False,
                  dropout=0.2)(img_input_4)
        
    concated = layers.concatenate([
        encoder_img_0,
        encoder_img_1,
        encoder_img_2,
        encoder_img_3,
        encoder_img_4,
        encoder_feat])
    
    rv = layers.RepeatVector(PREDICT_LENGTH)(concated)
    
    #lstm decoder
    decoder = layers.GRU(N_HIDDEN,
                  return_sequences=True,
                  stateful=False,
                  dropout=0.2)(rv)
    dense = layers.TimeDistributed(layers.Dense(3), input_shape=(PREDICT_LENGTH, None))(decoder)
    out = layers.Activation('linear')(dense)
    
    model = Model(inputs=[
        img_input_0,
        img_input_1,
        img_input_2,
        img_input_3,
        img_input_4,
        feat_input], outputs=[out])
    model.compile(loss='mse', optimizer=opt)
    
    print(model.summary())
    return model

In [ ]:
# Aggregated Training Error
model = build_model()

# Generators
training_generator = DataGenerator(train_folders)
val_generator = DataGenerator(val_folders)

checkpointer = ModelCheckpoint(filepath="/home/zg2309/model/{}.h5".format(MODEL_NAME), verbose=1, save_best_only=True)
# early_stopping_callback = EarlyStopping(monitor='val_loss', mode='auto')
history = model.fit_generator(generator=training_generator,
                    validation_data=val_generator,
                    use_multiprocessing=True,
                    epochs=EPOCHS,
                    verbose=1,
                    workers=6,
                    callbacks=[checkpointer])
# history = model.fit(train_X, train_y, validation_split=val_ratio, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpointer])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 10, 2048)]   0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 10, 2048)]   0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 10, 2048)]   0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 10, 2048)]   0                                            
______________________________________________________________________________________________

 31/281 [==>...........................] - ETA: 1:39 - loss: 0.1726
Epoch 00009: val_loss improved from 0.17964 to 0.17258, saving model to /home/zg2309/model/sslstm-image_epoch_2000_hidden_128_observe_10_predict_5.h5
281/281 [==============================] - 149s 529ms/step - loss: 0.1953 - val_loss: 0.1726

Epoch 10/2000
 31/281 [==>...........................] - ETA: 1:47 - loss: 0.1674
Epoch 00010: val_loss improved from 0.17258 to 0.16738, saving model to /home/zg2309/model/sslstm-image_epoch_2000_hidden_128_observe_10_predict_5.h5
281/281 [==============================] - 151s 539ms/step - loss: 0.1912 - val_loss: 0.1674
Epoch 11/2000
 31/281 [==>...........................] - ETA: 1:42 - loss: 0.1637
Epoch 00011: val_loss improved from 0.16738 to 0.16372, saving model to /home/zg2309/model/sslstm-image_epoch_2000_hidden_128_observe_10_predict_5.h5
281/281 [==============================] - 151s 537ms/step - loss: 0.1877 - val_loss: 0.1637
Epoch 12/2000
 31/281 [==>............

 31/281 [==>...........................] - ETA: 1:46 - loss: 0.1435
Epoch 00029: val_loss improved from 0.14405 to 0.14349, saving model to /home/zg2309/model/sslstm-image_epoch_2000_hidden_128_observe_10_predict_5.h5
281/281 [==============================] - 150s 535ms/step - loss: 0.1660 - val_loss: 0.1435
Epoch 30/2000
 31/281 [==>...........................] - ETA: 1:39 - loss: 0.1432Epoch 1/2000

Epoch 00030: val_loss improved from 0.14349 to 0.14315, saving model to /home/zg2309/model/sslstm-image_epoch_2000_hidden_128_observe_10_predict_5.h5
281/281 [==============================] - 152s 541ms/step - loss: 0.1636 - val_loss: 0.1432
Epoch 31/2000
 31/281 [==>...........................] - ETA: 1:45 - loss: 0.1456
Epoch 00031: val_loss did not improve from 0.14315
281/281 [==============================] - 150s 536ms/step - loss: 0.1640 - val_loss: 0.1456
Epoch 32/2000
 31/281 [==>...........................] - ETA: 1:44 - loss: 0.1409
Epoch 00032: val_loss improved from 0.14315

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 31/281 [==>...........................] - ETA: 1:44 - loss: 0.1393Epoch 1/200
Epoch 00039: val_loss did not improve from 0.13677
281/281 [==============================] - 151s 536ms/step - loss: 0.1571 - val_loss: 0.1393
Epoch 40/2000
 31/281 [==>...........................] - ETA: 1:45 - loss: 0.1346
Epoch 00040: val_loss improved from 0.13677 to 0.13458, saving model to /home/zg2309/model/sslstm-image_epoch_2000_hidden_128_observe_10_predict_5.h5
281/281 [==============================] - 152s 542ms/step - loss: 0.1570 - val_loss: 0.1346
Epoch 41/2000
 31/281 [==>...........................] - ETA: 1:47 - loss: 0.1304
Epoch 00041: val_loss improved from 0.13458 to 0.13044, saving model to /home/zg2309/model/sslstm-image_epoch_2000_hidden_128_observe_10_predict_5.h5
281/281 [==============================] - 154s 547ms/step - loss: 0.1564 - val_loss: 0.1304
Epoch 42/2000
 31/281 [==>...........................] - ETA: 1:42 - loss: 0.1349
Epoch 00042: val_loss did not improve from 0.

...

 31/281 [==>...........................] - ETA: 1:41 - loss: 0.0283
Epoch 00638: val_loss did not improve from 0.02811
281/281 [==============================] - 149s 530ms/step - loss: 0.0364 - val_loss: 0.0283
Epoch 639/2000
 31/281 [==>...........................] - ETA: 1:42 - loss: 0.0317
Epoch 00639: val_loss did not improve from 0.02811
281/281 [==============================] - 150s 534ms/step - loss: 0.0358 - val_loss: 0.0317
Epoch 640/2000
 31/281 [==>...........................] - ETA: 1:55 - loss: 0.0306
Epoch 00640: val_loss did not improve from 0.02811
281/281 [==============================] - 152s 541ms/step - loss: 0.0358 - val_loss: 0.0306
Epoch 641/2000
 31/281 [==>...........................] - ETA: 1:46 - loss: 0.0322
Epoch 00641: val_loss did not improve from 0.02811
281/281 [==============================] - 151s 537ms/step - loss: 0.0362 - val_loss: 0.0322
Epoch 642/2000
 31/281 [==>...........................] - ETA: 1:40 - loss: 0.0291
Epoch 00642: val_loss di

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 31/281 [==>...........................] - ETA: 1:39 - loss: 0.0296
Epoch 00657: val_loss did not improve from 0.02811
281/281 [==============================] - 150s 532ms/step - loss: 0.0357 - val_loss: 0.0296
Epoch 658/2000
 31/281 [==>...........................] - ETA: 1:48 - loss: 0.0309
Epoch 00658: val_loss did not improve from 0.02811
281/281 [==============================] - 151s 537ms/step - loss: 0.0353 - val_loss: 0.0309
Epoch 659/2000
 31/281 [==>...........................] - ETA: 1:47 - loss: 0.0285
Epoch 00659: val_loss did not improve from 0.02811
281/281 [==============================] - 153s 546ms/step - loss: 0.0361 - val_loss: 0.0285
Epoch 660/2000
 31/281 [==>...........................] - ETA: 1:45 - loss: 0.0298
Epoch 00660: val_loss did not improve from 0.02811
281/281 [==============================] - 154s 548ms/step - loss: 0.0358 - val_loss: 0.0298
Epoch 661/2000
 31/281 [==>...........................] - ETA: 1:47 - loss: 0.0276
Epoch 00661: val_loss im

Epoch 684/2000
 31/281 [==>...........................] - ETA: 1:48 - loss: 0.0300
Epoch 00684: val_loss did not improve from 0.02759
281/281 [==============================] - 150s 533ms/step - loss: 0.0347 - val_loss: 0.0300
Epoch 685/2000
 31/281 [==>...........................] - ETA: 1:47 - loss: 0.0292
Epoch 00685: val_loss did not improve from 0.02759
281/281 [==============================] - 154s 550ms/step - loss: 0.0347 - val_loss: 0.0292
Epoch 686/2000
 31/281 [==>...........................] - ETA: 1:44 - loss: 0.0292
Epoch 00686: val_loss did not improve from 0.02759
281/281 [==============================] - 152s 541ms/step - loss: 0.0348 - val_loss: 0.0292
Epoch 687/2000
 31/281 [==>...........................] - ETA: 1:44 - loss: 0.0286
Epoch 00687: val_loss did not improve from 0.02759
281/281 [==============================] - 148s 526ms/step - loss: 0.0350 - val_loss: 0.0286
Epoch 688/2000
 31/281 [==>...........................] - ETA: 1:40 - loss: 0.0309
Epoch 006

 31/281 [==>...........................] - ETA: 1:39 - loss: 0.0282
Epoch 00711: val_loss did not improve from 0.02735
281/281 [==============================] - 151s 538ms/step - loss: 0.0343 - val_loss: 0.0282
Epoch 712/2000
 31/281 [==>...........................] - ETA: 1:41 - loss: 0.0289
Epoch 00712: val_loss did not improve from 0.02735
281/281 [==============================] - 147s 524ms/step - loss: 0.0337 - val_loss: 0.0289
Epoch 713/2000
 31/281 [==>...........................] - ETA: 1:44 - loss: 0.0288
Epoch 00713: val_loss did not improve from 0.02735
281/281 [==============================] - 148s 525ms/step - loss: 0.0337 - val_loss: 0.0288
Epoch 714/2000
 31/281 [==>...........................] - ETA: 1:36 - loss: 0.0280
Epoch 00714: val_loss did not improve from 0.02735
281/281 [==============================] - 149s 529ms/step - loss: 0.0344 - val_loss: 0.0280
Epoch 715/2000
 31/281 [==>...........................] - ETA: 1:39 - loss: 0.0293
Epoch 00715: val_loss di

In [ ]:
# Plot training & validation loss values
fig = plt.gcf()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
fig.savefig('/home/zg2309/history/{}'.format(MODEL_NAME))
plt.close(fig)